<a href="https://colab.research.google.com/github/dbpedia/RDF2text-GAN/blob/master/Transformers/discriminator_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pre-train Discriminator before adverserial training

The discriminator is trained by concatenating the input and the output sequence of an rdf triple-text pair, and passing the concatenated sequence as a whole to the discriminator. The idea here is that the discriminator learns to discriminate between real rdf-txt pqired sequences and fake ones.



- Adapted from : Original Author:** [Apoorv Nandan](https://twitter.com/NandanApoorv)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import time
import numpy as np


In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

file_path = "/content/gdrive/My Drive/data/processed_graphs/eng/gat/f_data.txt"
test_path = "/content/gdrive/My Drive/data/processed_graphs/eng/gat/test_data.txt"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pretraining
from pretraining import *

In [ ]:
train_dataset, tokenizer_txt = create_generator_dataset(file_path)
X_train, X_test, y_train, y_test = create_discriminator_dataset(train_dataset)

Max source length :  133 
Max target length :  87
Max sequence size:  215
dataset shape:  (68704, 215) (68704,)


In [ ]:
k = lambda x : x.shape
list(map(k, [X_train, X_test, y_train, y_test]))

[(61833, 215), (6871, 215), (61833,), (6871,)]

In [ ]:
DATA_MAX_LEN=250

In [ ]:
from transformer_discriminator import *
discriminator = TransformerDiscriminator(tokenizer_txt.vocab_size+2, maxlen=DATA_MAX_LEN)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
token_and_position_embedding (None, 250, 32)           348000    
_________________________________________________________________
transformer_block (Transform (None, 250, 32)           6464      
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0     

## Train and Evaluate


In [ ]:
discriminator.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history = discriminator.fit(
    X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test)
)


Epoch 1/10


1932/1933 [============================>.] - ETA: 0s - loss: 0.6738 - accuracy: 0.5780WARNING:tensorflow:Model was constructed with shape (None, 250) for input Tensor("input_1:0", shape=(None, 250), dtype=float32), but it was called on an input with incompatible shape (None, 215).


1933/1933 [==============================] - 173s 90ms/step - loss: 0.6738 - accuracy: 0.5780 - val_loss: 0.6624 - val_accuracy: 0.6025
Epoch 2/10
1933/1933 [==============================] - 173s 90ms/step - loss: 0.6314 - accuracy: 0.6364 - val_loss: 0.5037 - val_accuracy: 0.7626
Epoch 3/10
1933/1933 [==============================] - 174s 90ms/step - loss: 0.4081 - accuracy: 0.8172 - val_loss: 0.3639 - val_accuracy: 0.8451
Epoch 4/10
1933/1933 [==============================] - 173s 90ms/step - loss: 0.3153 - accuracy: 0.8719 - val_loss: 0.3130 - val_accuracy: 0.8750
Epoch 5/10
1933/1933 [==============================] - 168s 87ms/step - loss: 0.2267 - accuracy: 0.9165 - val_loss: 0.2195 - val_accuracy: 0.9214
Epoch 6/10
1933/1933 [==============================] - 172s 89ms/step - loss: 0.1646 - accuracy: 0.9434 - val_loss: 0.2066 - val_accuracy: 0.9288
Epoch 7/10
1933/1933 [==============================] - 167s 87ms/step - loss: 0.1291 - accuracy: 0.9577 - val_loss: 0.1894 - val

In [ ]:
discriminator.save_weights('./discriminator_weights.h5')